<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_SAAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Informe Provincial Cuenca
##Descarga y limpeza de datos

Comenzamos cargando los datos de la carpeta de Google Drive en la que estén guardados (será necesario cambiar esta ruta en función de la ruta del ordenador).

Es necesario aceptar la conexión con la cuenta de Google Drive.

Importante y esencial, antes de tirar el código es necesario que la carpeta de Drive a la que se va a llamar tenga los archivos en formato xlsx así nombrados: 'CU_SAAF.xlsx', 'CU_AT.xlsx', 'CU_EMPLEO.xlsx', 'CU_LOG.xlsx' para que no haya errores a la hora de ejecutar este código.

**¡OJO! Primeros archivo con extensión xlsx y los de las zonas a clasificar, con extensión csv, en teoría, los csv no deberían modificarse (a menos que las normativas cambien y con ello las zonas de impacto se modifiquen).**

**Y todos las columnas deben tener los nombres definidos en el documento "Resumen de datos por servicio FASPAS" y en mayúsculas antes de ser subidos al programa.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: The domain policy has disabled Drive File Stream: https://support.google.com/a/answer/7496409

Se importan las librerías necesarias para la parte de carga de datos y su limpieza.

In [1]:
pip install unidecode

In [2]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [6]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/SAAF'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path_csv = '/content/drive/My Drive/PFG_FASPAS/Zonas_Prioritarias'
docs_csv = [f for f in os.listdir(folder_path_csv) if f.endswith('.csv')]
print(docs_csv)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/PFG_FASPAS/SAAF'

Creación de un diccionario que almacene los dataframes (.xlsx) contenidos en la carpeta de Cuenca.

Se irán cargando todos los archivos en un dataframe que se añade al diccionario con el nombre de dicho archivo como clave.

In [ ]:
dic_dataframes = {}
dic_zonas = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

for doc in docs_csv:
  entire_path = os.path.join(folder_path_csv, doc)
  df = pd.read_csv(entire_path)
  # Uso el nombre del archivo como clave
  dic_zonas[doc] = df

A continuación se debe observar que el dataframe cuente con al menos las columnas: 'AÑOS ATENCIÓN', 'FECHA NACIMIENTO' **en formato fecha**, 'LOCALIDAD', 'GÉNERO' y 'CP'

Ahora que sabemos cuáles son las claves, guardamos cada dataframe por separado para trabajar con todos ellos de manera individual, por el momento.

In [ ]:
# # Imprimir todas las claves
# for clave in dic_dataframes.keys():
#     print(clave)

In [ ]:
# CU_SAAF = dic_dataframes.get("CU_SAAF.xlsx")
# CU_AT = dic_dataframes.get("CU_AT.xlsx")
# CU_EMPLEO = dic_dataframes.get("CU_EMPLEO.xlsx")
# CU_LOG = dic_dataframes.get("CU_LOG.xlsx")
# # CU_SOCIOS = dic_dataframes.get("CU_SOCIOS.xlsx")

Vamos a limpiar cada uno de los dataframes en función de los datos que tenemos y vamos a normalizarlos para cuando los crucemos entre ellos.

Comenzamos con los datos de Servicio de Atención y Apoyo a Familias (SAAF).

Sacamos la información del dataset y observamos que de 77 entradas, el campo de 'GRADO DEPENDENCIA' tan solo tiene 1 registro, por lo que se procede a eliminarla. El 'GRADO DISCAPACIDAD' de momento lo dejamos ya que para el impacto territorial no lo vamos a usar.

In [29]:
# # CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
CU_SAAF = pd.read_excel('/content/CU_SAAF_PRUEBA.xlsx')
# CU_AT = pd.read_excel('/content/CU_AT.xlsx')
# CU_EMPLEO = pd.read_excel('/content/CU_EMPLEO.xlsx')
# CU_LOG = pd.read_excel('/content/CU_LOG.xlsx')
# # CU_SOCIOS = pd.read_excel('/content/CU_SOCIOS.xlsx')
mun_pri_clm = pd.read_csv('/content/municipios_prioritarios_clm.csv')

Primero se ponen todos los nombres de las columnas y todos los valores de los registros en mayúsculas y se eliminan todas las tildes contenidas (tanto en los registros de tipo texto como en las columnas), para uniformar el dataset.

In [39]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    # Convertir todas las cadenas en las columnas de objeto a mayúsculas
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.upper()

    # Convertir los nombres de las columnas a mayúsculas y sin tildes
    df.columns = [unidecode(col.upper()) for col in df.columns]

    # Seleccionar las columnas de texto
    text_columns = df.select_dtypes(include=['object']).columns

    # Aplicar unidecode solo a las columnas de texto
    df[text_columns] = df[text_columns].applymap(lambda x: unidecode(x) if isinstance(x, str) else x)

    # Devolver el DataFrame transformado
    return df

# Asegurarse de que todas las columnas necesarias existan en todos los dataframes, creándolas si es necesario (OJO: COLUMNAS NECESARIAS PARA TODOS LOS SERVICIOS)
# Limpieza de todas aquellas columnas no pedidas
def kept_columns(df):
  # Lista de columnas necesarias para hacer el impacto territorial en todos los dataframes
  all_needed_columns = ['FECHA NACIMIENTO', 'LOCALIDAD', 'GENERO', 'CP']
  for col in all_needed_columns:
    if col not in df.columns:
      df[col] = np.nan

  columns_to_keep = ['FECHA ALTA','FECHA DE ALTA', 'FECHA NACIMIENTO','FECHA DE NACIMIENTO','FECHA NAC' ,'LOCALIDAD', 'GENERO', 'CP', 'TIPO SORDERA', 'MOMENTO APARICION', 'MOMENTO DE APARICION DE LA SORDERA', '¿LLEVA AUDIFONO?', '¿LLEVA I.C.?', '¿LLEVA IMPLANTE DE TRONCO CEREBRAL?', '¿LLEVA IMPLANTE OSTEOINTEGRADO?', '¿LLEVA IMPLANTE DE OIDO MEDIO?', 'MODALIDAD COMUNICATIVA', 'MODALIDAD COMUNICATIVA EN CASA','MOD COM','GRADO DISCAPACIDAD', 'TIPO PROTESIS', 'TIPO DE PROTESIS','TIPO DE PROTESIS AUDITIVA', 'GRADO PERDIDA', 'GRADO DE PERDIDA']
  df = df[df.columns.intersection(columns_to_keep)]
  return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'FECHA DE ALTA': 'FECHA ALTA',
        'FECHA NAC': 'FECHA NACIMIENTO',
        'FECHA DE NACIMIENTO': 'FECHA NACIMIENTO',
        'MOMENTO DE APARICION DE LA SORDERA': 'MOMENTO APARICION',
        'TIPO DE PROTESIS AUDITIVA': 'TIPO PROTESIS',
        'TIPO DE PROTESIS': 'TIPO PROTESIS',
        'MOD COM': 'MODALIDAD COMUNICATIVA',
        'MODALIDAD COMUNICATIVA EN CASA': 'MODALIDAD COMUNICATIVA',
        'GRADO DE PERDIDA': 'GRADO PERDIDA'
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def CP_year(df):

  # df['FECHA NACIMIENTO'] = df['FECHA NACIMIENTO'].apply(lambda x: np.nan if str(x).isdigit() or x == '(ADULTO)' else x)
  # df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

  df['CP'] =df['CP'].astype('Int64')

  df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

  df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')

  CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()

  df = df[~CP_loc_filter]
  return df

**OJOOOOOOOO HAY QUE VER CÓMO PROCEDER PARA AQUELLOS REGISTROS CON DATOS NULOS EN LOS CAMPOS DE CP Y LOCALIDAD**

¿¿SI HAY UNO DE LOS DOS DEJAMOS EL REGISTRO, PERO SI NO HAY NADA EN AMBOS CAMPOS LOS ELIMINAMOS?? está justo arriba

Vamos a comenzar a limpiar los datos del Servicio de Asistencia y Apoyo a Familias (sAAF).

Lo primero que hacemos es eliminar aquellas columnas cuyos datos NO se hayan pedido, ya que aunque no se hayan podido recopilar todos los datos pedidos de todas las provincias, queremos que los datasets sean lo más homogéneos posibles.

Por otra parte, la columna 'FECHA NACIMIENTO' nos quedamos solo con la fecha (OJOOOO ¿O SOLO EL AÑO???? O AMBAS OPCIONES), eliminando la hora. Por defecto, Excel pone el CP como float, asi que lo pasamos a entero.

**YO NO QUITARÍA LA FECHA DE NACIMIENTO CONCRETA POR SI HAY QUE CONTEMPLAR QUE LOS DATOS DE ENTRADA TENGAN USUARIOS REPETIDOS, PERO PARA HACER ESTUDIO DE EDAD, USARÍA LA COLUMNA DE 'AÑO NACIMIENTO'. CONSULTAAAAAAAAAAAAAAAR.**

Si existe en el dataframe, como en este caso, aquellos que no tienen registro de 'TIPO PRÓTESIS' hay que examinar el motivo. Vamos a ver aquellos registros distintos que tienen todas las columnas. Vemos que en la columna 'TIPO PRÓTESIS' no hay ninguna salida que sea "NO TIENE" y preguntamos a FASPAS, tras su aprobación, procedemos a sustituir los valores nulos por 'NO TIENE'.

Se observa que la 'MODALIDAD COMUNICATIVA' no da ninguna información, pues todos los registros son "ORAL", sim embargo, la dejamos por si acaso en un futuro los registros cambiaran.

Se disminuye la complejidad de los datos de manera que solo nos quedamos con la categoría de 'AUD O PRÓTESIS' que indica si lleva o no alguno de los dispotivos posibles (¿Lleva implante de tronco cerebral/osteointegrado/de oído medio?) con independencia de cuál de estos sea.

In [31]:
# Creación de columna 'AUD O PRÓTESIS' en función del dataset de entrada
def determinar_si_o_no(row):
    # Lista de las columnas a verificar
    columns_to_meet = ['¿LLEVA AUDIFONO?', '¿LLEVA I.C.?', '¿LLEVA IMPLANTE DE TRONCO CEREBRAL?', '¿LLEVA IMPLANTE OSTEOINTEGRADO?', '¿LLEVA IMPLANTE DE OIDO MEDIO?']
    # Itera sobre cada columna especificada
    for col in columns_to_meet:
        # Verifica si el valor es NaN, "No" o "no tiene"
        if pd.isna(row[col]) or row[col] == 'NO' or row[col] == 'NO TIENE':
            return 'NO'
    # Si ninguna columna cumple la condición anterior, devuelve 'Sí'
    return 'SI'

def other_columns(df):
  if 'TIPO PROTESIS' in df.columns:
    df['TIPO PROTESIS'] = df['TIPO PROTESIS'].fillna('NO TIENE')
    df['AUD O PROTESIS'] = np.where(df['TIPO PROTESIS'] == 'NO TIENE', 'NO', 'SI')
    df.drop('TIPO PROTESIS', axis=1, inplace=True)

  elif ('¿LLEVA AUDIFONO?' and '¿LLEVA I.C.?' and '¿LLEVA IMPLANTE DE TRONCO CEREBRAL?' and '¿LLEVA IMPLANTE OSTEOINTEGRADO?' and '¿LLEVA IMPLANTE DE OIDO MEDIO?') in df.columns:
    df['AUD O PROTESIS'] = df.apply(determinar_si_o_no, axis=1)

  # if '¿LLEVA AUDIFONO?' in df.columns:
  #   si = ['SI', 'YES', 'S']
  #   for s in si:
  #     df['¿LLEVA AUDIFONO?'] = np.where(df['¿LLEVA AUDIFONO?'] == s, 'SI', 'NO')
  #     df['¿LLEVA I.C.?'] = np.where(df['¿LLEVA I.C.?'] == s, 'SI', 'NO')
  #     df['¿LLEVA IMPLANTE DE TRONCO CEREBRAL?'] = np.where(df['¿LLEVA IMPLANTE DE TRONCO CEREBRAL?'] == s, 'SI', 'NO')
  #     df['¿LLEVA IMPLANTE OSTEOINTEGRADO?'] = np.where(df['¿LLEVA IMPLANTE OSTEOINTEGRADO?'] == s, 'SI', 'NO')
  #     df['¿LLEVA IMPLANTE DE OIDO MEDIO?'] = np.where(df['¿LLEVA IMPLANTE DE OIDO?'] == s, 'SI', 'NO')

  #   # Creación de columna 'AUD O PRÓTESIS' y elimino las demás
  #   df['AUD O PROTESIS'] = df[['¿LLEVA AUDIFONO?' ,'¿LLEVA I.C.?', '¿LLEVA IMPLANTE DE TRONCO CEREBRAL?', '¿LLEVA IMPLANTE OSTEOINTEGRADO?', '¿LLEVA IMPLANTE DE OIDO MEDIO?']].apply(lambda x: x == 'SI').any(axis=1)
    df.drop(columns=['¿LLEVA AUDIFONO?' ,'¿LLEVA I.C.?', '¿LLEVA IMPLANTE DE TRONCO CEREBRAL?', '¿LLEVA IMPLANTE OSTEOINTEGRADO?', '¿LLEVA IMPLANTE DE OIDO MEDIO?'], inplace=True)

  if 'GENERO' in df.columns:
    # OJO: contemplar tb las diferentes formas de escribir TODAS LAS COLUMNAS 'MOMENTO APARICIÓN', HAY QUE NORMALIZAR TODO
    mujer = ['MUJER', 'HEMBRA', 'FEMENINO', 'FEM', 'F']
    hombre = ['HOMBRE', 'MACHO', 'MASCULINO', 'MASC', 'MAS']

    for m in mujer:
      df['GENERO'] = df['GENERO'].replace(m, 'M')

    for h in hombre:
      df['GENERO'] = df['GENERO'].replace(h, 'H')
  return df

Para contemplar que otros datos que se ingresen sean distintos, vamos a jugar con eliminar aquellas columnas que no sean esenciales para el estudio territorial, sino que nos den información más sociológica y sirvan para un estudio social; siempre y cuando el porcentaje de valores nulos sea mayor o igual que 1/3 del total de registros.

In [32]:
def delete_not_important_columns(df):
  # Añado 'AÑO ATENCIÓN' para SAAF
  needed_columns = ['AÑO ATENCION', 'FECHA NACIMIENTO', 'LOCALIDAD', 'GENERO', 'CP', 'YEAR NACIMIENTO']

  # Creación de la lista de columnas consideradas para posible eliminación
  social_columns = [col for col in df.columns if col not in needed_columns]

  # Porcentaje máximo de valores nulos permitido
  max_percentage = 1/3

  # Identifico columnas para eliminar
  del_columns = []
  for col in social_columns:
      if df[col].isnull().sum() / len(df) >= max_percentage:
          del_columns.append(col)

  # Eliminar las columnas identificadas
  df.drop(columns=del_columns, inplace=True)
  return df

Además, pondremos todos los registros de MUJER como M y HOMBRE como H. Esto lo realizaremos con todos los dataframes. Contemplamos que los registros que llegan pueden tener otros nombres en este campo.

In [33]:
def left_columns_norm(df):
  if 'TIPO SORDERA' in df.columns:
    # Lista con las posibles salidas permitidas
    strings_to_keep = ['NEUROSENSORIAL', 'CONDUCTIVA', 'MIXTA', 'TEL']

    # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
    df.loc[~df['TIPO SORDERA'].isin(strings_to_keep), 'TIPO SORDERA'] = 'NO REGISTRADO'

  if 'MOMENTO APARICION' in df.columns:
    strings_to_keep = ['PERILOCUTIVA', 'POSTLOCUTIVA', 'PRELOCUTIVA']

    # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
    df.loc[~df['MOMENTO APARICION'].isin(strings_to_keep), 'MOMENTO APARICION'] = 'NO REGISTRADO'

  if 'MODALIDAD COMUNICATIVA' in df.columns:
    strings_to_keep = ['SIGNO', 'LSE','SIGNOS', 'ORAL', 'BIMODAL', 'L.S.E.', 'L.S.E', 'SIGNOS NATURALES', 'ORAL,SIGNOS NATURALES', 'ORAL, SIGNOS NATURALES']

    # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
    df.loc[~df['MODALIDAD COMUNICATIVA'].isin(strings_to_keep), 'MODALIDAD COMUNICATIVA'] = 'NO REGISTRADO'

    # Normalizo todas las formas de llamar a las distintas modalidades comunicativas
    ORAL = ['ORAL', 'ORAL,SIGNOS NATURALES', 'ORAL, SIGNOS NATURALES']
    LSE = ['LSE', 'L.S.E.','L.S.E']
    SIGNOS = ['SIGNO', 'SIGNOS', 'SIGNOS NATURALES']

    for o in ORAL:
      df['MODALIDAD COMUNICATIVA'] = df['MODALIDAD COMUNICATIVA'].replace(o, 'ORAL')

    for l in LSE:
      df['MODALIDAD COMUNICATIVA'] = df['MODALIDAD COMUNICATIVA'].replace(l, 'LSE')

    for s in SIGNOS:
      df['MODALIDAD COMUNICATIVA'] = df['MODALIDAD COMUNICATIVA'].replace(s, 'SIGNOS')

  if 'GRADO PERDIDA' in df.columns:
    df['GRADO PERDIDA'] = df['GRADO PERDIDA'].astype(str)

    # Selecciono solo las 3 primeras letras (quitando la parte de DB si la tuviera)
    df['GRADO PERDIDA'] = df['GRADO PERDIDA'].str.slice(0, 3)

    strings_to_keep = ['DAP', 'DAM', 'DAS', 'DAL']

    df.loc[~df['GRADO PERDIDA'].isin(strings_to_keep), 'GRADO PERDIDA'] = 'NO REGISTRADO'

  if 'GRADO DISCAPACIDAD' in df.columns:
    df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].apply(lambda x: x / 100 if x > 1 else x)
    # OJOOOOOOOOOOOOOOO PREGUNTAR A OLGA SI LE PARECE BIEN O MEJOR PONERLO A CERO LAS QUE NO TENGAN REGISTRO!?!?
    df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].fillna(0)
    # df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].fillna(df['GRADO DISCAPACIDAD'].mean())
  return df

Aquí podemos ver la información del df, cuáles son las columnas que se tendrán en cuenta para este caso concreto y los diferentes datos que pueden tomar. Observamos que los únicos datos que pueden tener valores nulos son CP y LOCALIDAD, pero serán en registros distintos.

Se crea un filtro booleano que coge aquellos registros con ambos campos nulos y se lo aplicamos al dataframe. Ya que aquellos registros sin geolocalización no nos sirven para el objetivo principal que es saber el impacto que se tiene en las diferentes zonas para desarrollar un plan de ampliación territorial estratégico basado en el estudio estadístico.

In [12]:
# Suponiendo que 'dict_of_dfs' es tu diccionario de DataFrames
for df_key, df in dic_dataframes.items():
    dic_dataframes[df_key] = transform_df(df)
    dic_dataframes[df_key] = mapping_names(df)
    dic_dataframes[df_key] = kept_columns(df)
    dic_dataframes[df_key] = CP_year(df)
    dic_dataframes[df_key] = other_columns(df)
    dic_dataframes[df_key] = delete_not_important_columns(df)
    dic_dataframes[df_key] = left_columns_norm(df)

In [41]:
# CÓDIGO DE USO PARA EL CURRO
CU_SAAF = transform_df(CU_SAAF)
CU_SAAF = mapping_names(CU_SAAF)
CU_SAAF = kept_columns(CU_SAAF)
CU_SAAF = CP_year(CU_SAAF)
CU_SAAF = other_columns(CU_SAAF)
CU_SAAF = delete_not_important_columns(CU_SAAF)
CU_SAAF = left_columns_norm(CU_SAAF)

AttributeError: Can only use .dt accessor with datetimelike values

In [19]:
dic_dataframes['CU_SAAF.xlsx'].info()

NameError: name 'dic_dataframes' is not defined

In [40]:
# CÓDIGO DE USO PARA EL CURRO
CU_SAAF.info()
column_names = CU_SAAF.columns.tolist()

for column in column_names:
  unique_val = CU_SAAF[column].unique()
  print(unique_val)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   FECHA NACIMIENTO        4 non-null      object 
 1   GENERO                  77 non-null     object 
 2   LOCALIDAD               76 non-null     object 
 3   CP                      72 non-null     float64
 4   TIPO SORDERA            77 non-null     object 
 5   GRADO PERDIDA           77 non-null     object 
 6   MOMENTO APARICION       77 non-null     object 
 7   MODALIDAD COMUNICATIVA  77 non-null     object 
 8   GRADO DISCAPACIDAD      77 non-null     float64
dtypes: float64(2), object(7)
memory usage: 5.5+ KB
[nan '(ADULTO)']
['M' 'H']
['NOHALES' 'VALVERDE DEL JUCAR' 'CARBONERAS' 'CUENCA' 'MINGLANILLA'
 'POZORRUBIO' 'CHILLARON' 'CAMPILLO DE ALTOBUEY' 'TARANCON'
 'VILLAMAYOR DE SANTIAGO' 'VALERA DE ABAJO' 'ATALAYA DE CANAVATE' 'CANETE'
 'MOTA DEL CUERVO' 'MOTILLA DEL P

In [ ]:
column_names = dic_dataframes['CU_SAAF.xlsx'].columns.tolist()

for column in column_names:
  unique_val = dic_dataframes['CU_SAAF.xlsx'][column].unique()
  print(unique_val)

In [ ]:
df_comun = pd.merge(dic_dataframes['CU_SAAF.xlsx'], mun_pri_clm, left_on='LOCALIDAD', right_on='ZONA PRIORITARIA')
df_comun

Vemos el impacto que tiene en las zonas prioritarias.

In [ ]:
# Calcular el conteo de registros para cada categoría
cat_count = df_comun['LOCALIDAD'].value_counts()

# Crear un gráfico de barras
cat_count.sort_index().plot(kind='bar')

# Personalización adicional
plt.title('SAAF: Impacto en Zonas por Zonas Prioritarias')
plt.xlabel('ZONA PRIORITARIA')
plt.ylabel('Número de Registros')
plt.xticks(rotation=45)  # Rota las etiquetas del eje X para mejor lectura
plt.show()

In [ ]:
print("En este caso concreto las localidades con impacto dentro de las que se incluyen en las zonas prioritarias son: \n\nLocalidad Nº Registros")
print(cat_count)
# DEFINIR QUÉ SE CONSIDERA IMPACTO FUERTE/IMPACTO NORMAL/IMPACTO LEVE
# Iterar sobre la serie y mostrar solo las localidades con un valor mayor que 10
for localidad, valor in cat_count.items():
    if valor > 10:
      print(f"La Localidad de {localidad} tiene un impacto fuerte, ya que cuenta con {valor} registros.")
    elif valor > 5:
      print(f"La Localidad de {localidad} tiene un impacto medio, ya que cuenta con {valor} registros.")
    else:
      if valor > 1:
        print(f"La Localidad de {localidad} tiene un impacto leve, ya que cuenta con {valor} registros.")
      else:
        print(f"La Localidad de {localidad} tiene un impacto leve, ya que cuenta con {valor} registro.")